In [ ]:
from glob import glob
import logging
import os
from pprint import pprint
from time import time
from tqdm.notebook import tqdm

import tensorflow as tf

from ssd.common.callbacks import CallbackBuilder
from ssd.common.distribute import get_strategy
from ssd.common.config import load_config
from ssd.common.viz_utils import draw_boxes_cv2, imshow
from ssd.data.dataset_builder import DatasetBuilder
from ssd.losses.multibox_loss import MultiBoxLoss
from ssd.models.ssd_model import SSDModel

logger = tf.get_logger()
logger.setLevel(logging.INFO)

logger.info('version: {}'.format(tf.__version__))

In [ ]:
!python ssd/scripts/calculate_feature_shapes.py --image_height 480 --image_width 480 --num_feature_maps 6
# !python ssd/scripts/calculate_scales.py -n 7 --s_first 0.1 --smin 0.2 --smax 0.95
# !python check_matching.py

In [ ]:
config = load_config('ssd/cfg/sku110k-50.yaml')
config['use_tpu'] = False
config['cache_dataset_in_memory'] = False
config['batch_size'] = 64

strategy = get_strategy(config)

epochs = config['epochs']

lr = config['base_lr']
lr = config['base_lr']
lr = lr if not config['scale_lr'] else lr * strategy.num_replicas_in_sync

batch_size = config['batch_size']
batch_size = batch_size if not config['scale_batch_size'] else batch_size * strategy.num_replicas_in_sync
config['batch_size'] = batch_size

train_steps = config['train_images'] // config['batch_size']
val_steps = config['val_images'] // config['batch_size']

print('\n')
pprint(config, width=120, compact=True)

In [ ]:
with strategy.scope():
#     train_dataset = DatasetBuilder('train', config)
    val_dataset = DatasetBuilder('val', config)
    test_dataset = DatasetBuilder('test', config)

    loss_fn = MultiBoxLoss(config)
    optimizer = tf.optimizers.Adam(learning_rate=lr)
    callbacks_list = CallbackBuilder('test_run', config).get_callbacks()
    
    model = SSDModel(config)
    model.compile(loss_fn=loss_fn, optimizer=optimizer)

In [ ]:
with strategy.scope():
    latest_checkpoint = tf.train.latest_checkpoint(os.path.join(config['model_dir'] , 'best_weights'))
    logger.info('Loading weights from {}'.format(latest_checkpoint))
    model.load_weights(latest_checkpoint)

In [ ]:
for images, _ in test_dataset.dataset.take(1):
    for i in tqdm(range(images.shape[0])):

        image = images[i]
        detections = model.get_detections(image[None, ...])
        if 'resnet' in config['backbone']:
            image = image * 127.5 + 127.5
        
        categories = [config['classes'][cls_id] for cls_id in detections['cls_ids'].numpy()]
        image = draw_boxes_cv2(image, detections['boxes'], categories)
        imshow(image)

In [ ]:
image_path = 'kroger_apexawards_retail_coldspring.jpg'

image = tf.io.read_file(image_path)
image = tf.image.decode_image(image)
image = tf.image.resize(image, [config['image_height'], config['image_width']])

if 'resnet' in config['backbone']:
    image_preprocessed = (tf.cast(image, dtype=tf.float32) - 127.5) / 127.5
else:
    image_preprocessed = image

s = time()
detections = model.get_detections(image_preprocessed[None, ...])
e = time()
logger.info('Inference time: {:.3f}'.format(e - s))
categories = [config['classes'][cls_id] for cls_id in detections['cls_ids']]
image = draw_boxes_cv2(image, detections['boxes'], categories, thickness=1)
imshow(image)
